In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

import torch
torch.__version__, torch.version.cuda

('1.13.1+cu117', '11.7')

In [3]:
from src import models, data, lens, functional
from src.utils import experiment_utils

import logging
from src.utils import logging_utils
logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

In [4]:
device = "cuda:0"
mt = models.load_model("mamba-3b", device=device, fp16=False)
# mt = models.load_model("gptj", device=device, fp16=True)

2024-02-06 16:12:04 src.models INFO     loading EleutherAI/gpt-j-6B (device=cuda:0, fp16=True)
2024-02-06 16:12:04 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443


2024-02-06 16:12:04 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /EleutherAI/gpt-j-6B/resolve/float16/config.json HTTP/1.1" 307 0
2024-02-06 16:12:04 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /EleutherAI/gpt-j-6b/resolve/float16/config.json HTTP/1.1" 200 0
2024-02-06 16:12:21 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /EleutherAI/gpt-j-6B/resolve/float16/generation_config.json HTTP/1.1" 307 0
2024-02-06 16:12:21 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /EleutherAI/gpt-j-6b/resolve/float16/generation_config.json HTTP/1.1" 404 0
2024-02-06 16:12:24 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /EleutherAI/gpt-j-6B/resolve/main/tokenizer_config.json HTTP/1.1" 307 0
2024-02-06 16:12:24 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /EleutherAI/gpt-j-6b/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
2024-02-06 16:12:24 src.models INFO     dtype: torch.float16, de

In [5]:
dataset = data.load_dataset()

# relation_names = [r.name for r in dataset.relations]
# relation_options = Menu(choices = relation_names, value = relation_names)
# show(relation_options) # !caution: tested in a juputer-notebook. baukit visualizations are not supported in vscode.

2024-02-06 16:12:54 src.data DEBUG    no paths provided, using default data dir: /home/local_arnab/Codes/relations/notebooks/../data
2024-02-06 16:12:54 src.data DEBUG    /home/local_arnab/Codes/relations/notebooks/../data is directory, globbing for json files...
2024-02-06 16:12:54 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/characteristic_gender.json
2024-02-06 16:12:54 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/degree_gender.json
2024-02-06 16:12:54 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/name_birthplace.json
2024-02-06 16:12:54 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/name_gender.json
2024-02-06 16:12:54 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/name_religion.json
2024-02-06 16:12:54 src.data DEBUG    found relation 

In [6]:
relation_name = "country capital city"
relation = dataset.filter(relation_names=[relation_name])[0]
print(f"{relation.name} -- {len(relation.samples)} samples")
print("------------------------------------------------------")

experiment_utils.set_seed(12345) # set seed to a constant value for sampling consistency
train, test = relation.split(5)
print("\n".join([sample.__str__() for sample in train.samples]))

2024-02-06 16:12:56 src.data DEBUG    filtering to only relations: ['country capital city']
country capital city -- 24 samples
------------------------------------------------------
2024-02-06 16:12:56 src.utils.experiment_utils INFO     setting all seeds to 12345
China -> Beijing
Japan -> Tokyo
Italy -> Rome
Brazil -> Bras\u00edlia
Turkey -> Ankara


In [7]:
################### hparams ###################
layer = 20
beta = 8
###############################################

In [8]:
# from src.operators import JacobianIclMeanEstimator
# experiment_utils.set_seed(12345)

# estimator = JacobianIclMeanEstimator(
#     mt = mt, 
#     h_layer = layer,
#     beta = beta
# )
# operator = estimator(
#     relation.set(
#         samples=train.samples, 
#     )
# )

In [9]:
mt.name

'gptj'

In [10]:
import os
root_path = "../results/cached_o1_approxes"
path = os.path.join(
    root_path,
    mt.name,
    relation_name.lower().replace(" ", "_"),
    str(layer),
)

approxes = []
for cached_file in os.listdir(path):
    approx = functional.load_cached_linear_operator(file_path = os.path.join(path, cached_file))
    approxes.append(approx)


In [22]:
weight = torch.stack([approx.weight for approx in approxes]).mean(dim=0)
bias = torch.stack([approx.bias for approx in approxes]).mean(dim=0)

prompt_template = relation.prompt_templates[0]

prompt_template_icl = functional.make_prompt(
    mt = mt,
    prompt_template=prompt_template,
    subject="{}",
    examples = [
        data.RelationSample.from_dict(approx.metadata["sample"]) 
        for approx in approxes
    ][:min(3, len(approxes))],
)

print(prompt_template_icl)

<|endoftext|> The capital city of India is New Delhi
The capital city of Egypt is Cairo
The capital city of France is Paris
The capital city of {} is


In [25]:
from src.operators import LinearRelationOperator
operator = LinearRelationOperator(
    mt = mt,
    weight = weight,
    bias = bias,
    h_layer = approxes[0].h_layer,
    z_layer = approxes[0].z_layer,
    beta = 5,
    prompt_template = prompt_template_icl,
)

In [118]:
from src.lens import logit_lens
from src import models

# logit_lens(mt = mt, h = operator.metadata["Jh"][0].to(models.determine_device(mt)) + operator.bias)

In [119]:
from src.functional import predict_next_token

predict_next_token(
    mt = mt, 
    prompt = mt.tokenizer.eos_token + " The capital of {} is".format("France"),
    # prompt = mt.tokenizer.eos_token + " The superlative of {} is".format("good"),
)

[[PredictedToken(token=' Paris', prob=0.3467943072319031),
  PredictedToken(token=' a', prob=0.07820762693881989),
  PredictedToken(token=' the', prob=0.05906867980957031),
  PredictedToken(token=' located', prob=0.05243445932865143),
  PredictedToken(token=' also', prob=0.025281280279159546)]]

# Checking $faithfulness$

In [120]:
test = functional.filter_relation_samples_based_on_provided_fewshots(
    mt=mt, test_relation=test, prompt_template=operator.prompt_template, batch_size=4
)

2024-02-06 15:57:42 src.functional DEBUG    filtering for knowns using prompt "<|endoftext|> The capital city of China is Beijing
The capital city of Japan is Tokyo
The capital city of Italy is Rome
The capital city of Brazil is Bras\u00edlia
The capital city of Turkey is Ankara
The capital city of {} is"


2024-02-06 15:57:43 src.functional DEBUG    sample.subject='Argentina', sample.object='Buenos Aires', predicted=' Buenos' (p=0.940), known=(✓)
2024-02-06 15:57:43 src.functional DEBUG    sample.subject='Australia', sample.object='Canberra', predicted=' Can' (p=0.880), known=(✓)
2024-02-06 15:57:43 src.functional DEBUG    sample.subject='Canada', sample.object='Ottawa', predicted=' Ottawa' (p=0.923), known=(✓)
2024-02-06 15:57:43 src.functional DEBUG    sample.subject='Chile', sample.object='Santiago', predicted=' Santiago' (p=0.956), known=(✓)
2024-02-06 15:57:43 src.functional DEBUG    sample.subject='Colombia', sample.object='Bogot\\u00e1', predicted=' Bog' (p=0.971), known=(✓)
2024-02-06 15:57:43 src.functional DEBUG    sample.subject='Egypt', sample.object='Cairo', predicted=' Cairo' (p=0.973), known=(✓)
2024-02-06 15:57:43 src.functional DEBUG    sample.subject='France', sample.object='Paris', predicted=' Paris' (p=0.984), known=(✓)
2024-02-06 15:57:43 src.functional DEBUG    samp

In [121]:
sample = [s for s in test.samples if s.subject == "Canada"][0]
print(sample)
operator(subject = sample.subject).predictions

Canada -> Ottawa
2024-02-06 15:57:44 src.operators DEBUG    computing h from prompt "<|endoftext|> The capital city of China is Beijing
The capital city of Japan is Tokyo
The capital city of Italy is Rome
The capital city of Brazil is Bras\u00edlia
The capital city of Turkey is Ankara
The capital city of Canada is"


[PredictedToken(token=' Ottawa', prob=0.5542775988578796),
 PredictedToken(token=' Capital', prob=0.11941499263048172),
 PredictedToken(token=' Washington', prob=0.10894157737493515),
 PredictedToken(token=' Can', prob=0.0698302760720253),
 PredictedToken(token=' capital', prob=0.0381852351129055)]

In [122]:
hs_and_zs = functional.compute_hs_and_zs(
    mt = mt,
    prompt_template = operator.prompt_template,
    subjects = [sample.subject],
    h_layer= operator.h_layer,
)
h = hs_and_zs.h_by_subj[sample.subject]

## Approximating LM computation $F$ as an affine transformation

### $$ F(\mathbf{s}, c_r) \approx \beta \, W_r \mathbf{s} + b_r $$

In [123]:
z = 5 * (operator.weight @ h) + operator.bias

lens.logit_lens(
    mt = mt,
    h = z,
    get_proba = True
)

([(' Ottawa', 0.554),
  (' Capital', 0.119),
  (' Washington', 0.109),
  (' Can', 0.07),
  (' capital', 0.038),
  ('\n', 0.024),
  (' ', 0.016),
  (' the', 0.012),
  (' -', 0.008),
  (' in', 0.007)],
 {})

In [124]:
correct = 0
wrong = 0
for sample in test.samples:
    predictions = operator(subject = sample.subject).predictions
    known_flag = functional.is_nontrivial_prefix(
        prediction=predictions[0].token, target=sample.object
    )
    print(f"{sample.subject=}, {sample.object=}, ", end="")
    print(f'predicted="{functional.format_whitespace(predictions[0].token)}", (p={predictions[0].prob}), known=({functional.get_tick_marker(known_flag)})')
    
    correct += known_flag
    wrong += not known_flag
    
faithfulness = correct/(correct + wrong)

print("------------------------------------------------------------")
print(f"Faithfulness (@1) = {faithfulness}")
print("------------------------------------------------------------")

2024-02-06 15:57:47 src.operators DEBUG    computing h from prompt "<|endoftext|> The capital city of China is Beijing
The capital city of Japan is Tokyo
The capital city of Italy is Rome
The capital city of Brazil is Bras\u00edlia
The capital city of Turkey is Ankara
The capital city of Argentina is"
sample.subject='Argentina', sample.object='Buenos Aires', predicted=" Buenos", (p=0.7568550109863281), known=(✓)
2024-02-06 15:57:47 src.operators DEBUG    computing h from prompt "<|endoftext|> The capital city of China is Beijing
The capital city of Japan is Tokyo
The capital city of Italy is Rome
The capital city of Brazil is Bras\u00edlia
The capital city of Turkey is Ankara
The capital city of Australia is"
sample.subject='Australia', sample.object='Canberra', predicted=" Sydney", (p=0.5946000218391418), known=(✗)
2024-02-06 15:57:47 src.operators DEBUG    computing h from prompt "<|endoftext|> The capital city of China is Beijing
The capital city of Japan is Tokyo
The capital city o

# $causality$

In [125]:
################### hparams ###################
rank = 100
###############################################

In [126]:
experiment_utils.set_seed(12345) # set seed to a constant value for sampling consistency
test_targets = functional.random_edit_targets(test.samples)

2024-02-06 15:57:50 src.utils.experiment_utils INFO     setting all seeds to 12345


## setup

In [127]:
source = test.samples[0]
target = test_targets[source]

f"Changing the mapping ({source}) to ({source.subject} -> {target.object})"

'Changing the mapping (Argentina -> Buenos Aires) to (Argentina -> Riyadh)'

### Calculate $\Delta \mathbf{s}$ such that $\mathbf{s} + \Delta \mathbf{s} \approx \mathbf{s}'$

<p align="center">
    <img align="center" src="../causality_crop" style="width:80%;"/>
</p>

Under the relation $r =\, $*plays the instrument*, and given the subject $s =\, $*Miles Davis*, the model will predict $o =\, $*trumpet* **(a)**; and given the subject $s' =\, $*Cat Stevens*, the model will now predict $o' =\, $*guiter* **(b)**. 

If the computation from $\mathbf{s}$ to $\mathbf{o}$ is well-approximated by $operator$ parameterized by $W_r$ and $b_r$ **(c)**, then $\Delta{\mathbf{s}}$ **(d)** should tell us the direction of change from $\mathbf{s}$ to $\mathbf{s}'$. Thus, $\tilde{\mathbf{s}}=\mathbf{s}+\Delta\mathbf{s}$ would be an approximation of $\mathbf{s}'$ and patching $\tilde{\mathbf{s}}$ in place of $\mathbf{s}$ should change the prediction to $o'$ = *guitar* 

In [128]:
def get_delta_s(
    operator, 
    source_subject, 
    target_subject,
    rank = 100,
    fix_latent_norm = None, # if set, will fix the norms of z_source and z_target
):
    w_p_inv = functional.low_rank_pinv(
        matrix = operator.weight,
        rank=rank,
    )
    hs_and_zs = functional.compute_hs_and_zs(
        mt = mt,
        prompt_template = operator.prompt_template,
        subjects = [source_subject, target_subject],
        h_layer= operator.h_layer,
        z_layer=-1,
    )

    z_source = hs_and_zs.z_by_subj[source_subject]
    z_target = hs_and_zs.z_by_subj[target_subject]
    
    z_source *= fix_latent_norm / z_source.norm() if fix_latent_norm is not None else 1.0
    z_target *= z_source.norm() / z_target.norm() if fix_latent_norm is not None else 1.0

    delta_s = w_p_inv @  (z_target.squeeze() - z_source.squeeze())

    return delta_s, hs_and_zs

delta_s, hs_and_zs = get_delta_s(
    operator = operator,
    source_subject = source.subject,
    target_subject = target.subject,
    rank = rank
)

In [129]:
import baukit

def get_intervention(h, int_layer, subj_idx):
    def edit_output(output, layer):
        if(layer != int_layer):
            return output
        functional.untuple(output)[:, subj_idx] = h 
        return output
    return edit_output

prompt = operator.prompt_template.format(source.subject)

h_index, inputs = functional.find_subject_token_index(
    mt=mt,
    prompt=prompt,
    subject=source.subject,
)

h_layer, z_layer = models.determine_layer_paths(model = mt, layers = [layer, -1])

with baukit.TraceDict(
    mt.model, layers = [h_layer, z_layer],
    edit_output=get_intervention(
#         h = hs_and_zs.h_by_subj[source.subject],         # let the computation proceed as usual
        h = hs_and_zs.h_by_subj[source.subject] + delta_s, # replace s with s + delta_s
        int_layer = h_layer, 
        subj_idx = h_index
    )
) as traces:
    outputs = mt(
        input_ids = inputs.input_ids,
        attention_mask = inputs.attention_mask,
    )

logits = outputs.logits[0][-1] if hasattr(outputs, "logits") else outputs[0][-1]
lens.interpret_logits(
    mt = mt, 
    logits = logits, 
    get_proba=True
)

[(' R', 0.473),
 (' Saudi', 0.06),
 (' S', 0.05),
 (' Me', 0.028),
 ('\n', 0.023),
 (' K', 0.02),
 (' Jed', 0.02),
 (' \\', 0.011),
 (' _____', 0.01),
 (' Baghdad', 0.008)]

## Measuring causality

In [130]:
from src.editors import LowRankPInvEditor

svd = torch.svd(operator.weight.float())
editor = LowRankPInvEditor(
    lre=operator,
    rank=rank,
    svd=svd,
)

In [131]:
# precomputing latents to speed things up
hs_and_zs = functional.compute_hs_and_zs(
    mt = mt,
    prompt_template = operator.prompt_template,
    subjects = [sample.subject for sample in test.samples],
    h_layer= operator.h_layer,
    z_layer=-1,
    batch_size = 2
)

success = 0
fails = 0

for sample in test.samples:
    target = test_targets.get(sample)
    assert target is not None
    edit_result = editor(
        subject = sample.subject,
        target = target.subject
    )
    
    success_flag = functional.is_nontrivial_prefix(
        prediction=edit_result.predicted_tokens[0].token, target=target.object
    )
    
    print(f"Mapping {sample.subject} -> {target.object} | edit result={edit_result.predicted_tokens[0]} | success=({functional.get_tick_marker(success_flag)})")
    
    success += success_flag
    fails += not success_flag
    
causality = success / (success + fails)

print("------------------------------------------------------------")
print(f"Causality (@1) = {causality}")
print("------------------------------------------------------------")

2024-02-06 15:57:58 src.editors DEBUG    computing low-rank pinv (rel=<|endoftext|> The capital city of China is Beijing
The capital city of Japan is Tokyo
The capital city of Italy is Rome
The capital city of Brazil is Bras\u00edlia
The capital city of Turkey is Ankara
The capital city of {} is)
Mapping Argentina -> Riyadh | edit result=' R' (p=0.359) | success=(✓)
Mapping Australia -> Buenos Aires | edit result=' Buenos' (p=0.822) | success=(✓)
Mapping Canada -> Abuja | edit result=' Lag' (p=0.731) | success=(✗)
Mapping Chile -> Lima | edit result=' L' (p=0.792) | success=(✓)
Mapping Colombia -> Berlin | edit result=' Berlin' (p=0.777) | success=(✓)
Mapping Egypt -> Mexico City | edit result=' Mexico' (p=0.922) | success=(✓)
Mapping France -> Riyadh | edit result=' R' (p=0.751) | success=(✓)
Mapping Germany -> Cairo | edit result=' Cairo' (p=0.915) | success=(✓)
Mapping India -> Lima | edit result=' L' (p=0.926) | success=(✓)
Mapping Mexico -> Santiago | edit result=' Val' (p=0.313) 

In [49]:
from src.functional import save_linear_operator

save_linear_operator(
    approx = operator,
    file_name = "lre_capital",
    path = "cached"
)

In [50]:
operator_loaded = functional.load_cached_linear_operator(mt = mt, file_path = "cached/lre_capital.npz")

## Test

In [ ]:
prompt = mt.tokenizer.eos_token + " Michael Jordan professionally played the sport of"
tokenized = mt.tokenizer(prompt, return_tensors="pt").to(device)
models.determine_layer_paths(mt)[layer]

In [ ]:
import baukit

layer_out = models.determine_layer_paths(mt)[layer]
mixer_out = models.determine_layer_paths(mt)[layer] + ".mixer"
layer_in = models.determine_layer_paths(mt)[layer+1]
final_layer = models.determine_layer_paths(mt)[-1]
final_mixer = models.determine_layer_paths(mt)[-1] + ".mixer"

with baukit.TraceDict(
    module=mt.model,
    layers=[
        layer_out,
        mixer_out,
        layer_in,
        final_mixer,
        final_layer,
        "backbone"
    ],
    retain_input=True,
) as traces:
    output = mt(**tokenized)

In [ ]:
traces["backbone"].output

In [ ]:
traces[final_layer].output

In [ ]:
from torch import nn

class RMSNorm(nn.Module):
    def __init__(self,
                 d_model: int,
                 eps: float = 1e-5):
        super().__init__()
        self.eps = eps
        self.weight = baukit.get_module(mt.model, "backbone.norm_f").weight


    def forward(self, x):
        output = x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps) * self.weight

        return output

custom_rms = RMSNorm(d_model = models.determine_hidden_size(mt))
block_output, residual = traces[final_layer].output
backbone_output = custom_rms(block_output + residual)

backbone_output

In [ ]:
torch.allclose(traces[mixer_out].output, traces[layer_out].output[0])

In [ ]:
baukit.get_module(mt.model, "backbone.norm_f").bias

In [ ]:
hasattr(mt.model, "backbone")